In [1]:
%%time
# Program to pickle historical data for SD of nse underlyings

# STATUS: Completed
# Runtime: 12 mins (for a new dataset)

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

import pandas as pd
import numpy as np
import datetime
import os

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'
default_duration = 365 # days

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

#***      Function to get historical data     *****
#___________________________________________________
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)

    # Make the dataframe
    cols=['ibSymbol', 'D', 'O', 'H', 'L', 'C', 'Avg', 'Vol']
    df = pd.DataFrame([(contract.symbol, h.date, h.open, h.high, h.low, 
                       h.close, h.average, h.volume) 
                      for h in hist], columns=cols)
    return df

#*****    Get the symbols lists    ****
#_____________________________________*
symbols = list(pd.read_pickle(datapath+'df_underlying.pkl')['ibSymbol'])

indexes = list(pd.read_pickle(datapath+'df_nse_idx_symbols.pkl')['ibSymbol'])
equities = list(pd.read_pickle(datapath+'df_nse_eq_symbols.pkl')['ibSymbol'])

#*****    Qualify the contracts    ****
#_____________________________________*

# make the contracts and qualify them for symbols in underlying.pkl
contracts = [Index(symbol=s, exchange=market) 
 if s in indexes 
 else Stock(symbol=s, exchange=market) 
 for s in symbols]

qual_contracts = ib.qualifyContracts(*contracts)

#***            FIRST-RUN OHLC              ***
#______________________________________________

# Check if pickle file exists
f_exists = os.path.isfile(datapath+r'df_ohlc.pkl')

# Generate the entire pickle if it doesn't exist
# Takes about 8 mins for 365 days for 207 scrips

if not f_exists:
    df = [get_hist(contract=c, duration=365) for c in contracts]

    # concatenate the list of dataframes and pickle
    df1 = pd.concat(df).reset_index(drop=True)
    df1.to_pickle(datapath+r'df_ohlc.pkl')

#*****    Append OHLC for missing dates   ****
#_____________________________________________

# Get pickled data from disk
df_raw = pd.read_pickle(datapath+r'df_ohlc.pkl')
df = df_raw # initialize df

if f_exists:
    
    '''Missing OHLC dates / symbols to be added to the pickle'''
    
    #...........   Missing dates   .......
    #.....................................

    # Get the last date of the pickle 
    last_dt = df_raw.loc[(df_raw.ibSymbol == df_raw.ibSymbol[0]), 'D'].max()

    # Get the latest date of ib.reqHistoricalData
    c = Stock(symbol=df_raw.ibSymbol[0], exchange=market) # first symbol's contract

    latest_dt = ib.reqHistoricalData(contract=c,
                                     endDateTime= '', durationStr='1 D', barSizeSetting='1 day', 
                                     whatToShow='Trades', useRTH=True)[0].date

    delta_days = (latest_dt - last_dt).days    # additonal days required

    #.....      Correct the duration string    ....
    #..............................................
    if delta_days > 0:
        duration = str(delta_days)+ ' D'
        dates_list = [d.date for d in ib.reqHistoricalData(contract=c, endDateTime='',
                                   durationStr=duration, barSizeSetting='1 day', 
                                   whatToShow='Trades', useRTH=True)]

        # Correct delta days duration to points of data available.
        # This would be the duration string for ib.reqHistoricalData delta
        duration1 = len([d for d in dates_list if d >= last_dt])

        delta = [get_hist(contract=c, duration=duration1) for c in contracts]

        df_delta = pd.concat(delta).reset_index(drop=True)

        # Append to create the new data frame
        df = pd.concat(objs=[df_raw, df_delta], 
                           axis=0).sort_values(['ibSymbol', 'D']).reset_index(drop=True)
        

    #......... Missing Symbols handling ......
    #.........................................

    # Get the missing symbols in the pickle
    missing_syms = list(set([c.symbol for c in contracts]) - set(df_raw.ibSymbol.unique()))

    # If symbols are missing
    if missing_syms:
        missing_contracts = [c for c in contracts if c.symbol in missing_syms]
        missing = [get_hist(contract=c, duration=default_duration) for c in missing_contracts]
        df_missing = pd.concat(missing).reset_index(drop=True)

        # Add it to df
        df = pd.concat(objs=[df, df_missing], 
                            axis=0).sort_values(['ibSymbol', 'D']).reset_index(drop=True)

    # Overwrite the pickled file
    df.to_pickle(datapath+r'df_ohlc.pkl')

# Close the connection
ib.disconnect()

API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 4001)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 10061] Connect call failed ('127.0.0.1', 4001)